In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats import pearsonr

In [9]:
train_data = pd.read_csv("F:/code/prediction/数据文件(处理)/train.csv")
test_data = pd.read_csv("F:/code/prediction/数据文件(处理)/test.csv")

In [10]:
train_data.head()

,pt_in,pt_out,tt_in,tt_out,vf_td_s,vf_td,vf_su,pt_in_s,pt_out_s,tt_in_s,tt_out_s,vf_su_s,vf_fr,vf_fr_s
0,30.0,393.30,410.23,151.29,758.41,744.27,758.99,983.55,126.24,754.14,877.68,614.00,568.65,761.89
1,281.0,794.98,693.25,200.28,637.22,598.88,241.46,650.26,449.09,861.50,884.73,831.12,575.51,41.33
2,508.0,13.18,52.58,563.08,429.49,243.98,125.21,507.81,0.85,235.79,30.84,737.52,196.63,448.91
3,653.0,839.04,189.49,787.80,848.57,688.68,240.81,386.80,819.05,437.38,683.30,985.58,887.72,797.82
4,948.0,486.17,504.42,705.37,442.90,627.17,820.26,24.58,979.91,545.55,841.89,606.88,102.55,747.89


In [11]:
test_data.head()

,pt_in,pt_out,tt_in,tt_out,vf_td_s,vf_td,vf_su,pt_in_s,pt_out_s,tt_in_s,tt_out_s,vf_su_s
0,928.0,696.36,335.77,423.94,803.92,450.25,311.88,32.05,79.36,380.44,439.84,269.50
1,631.0,342.60,788.24,545.24,648.39,243.78,220.40,559.95,189.90,570.57,777.66,321.26
2,683.0,181.68,877.73,84.23,754.15,768.92,10.62,803.87,205.36,837.05,855.71,811.01
3,515.0,110.45,793.65,20.51,898.90,689.97,722.86,856.63,104.52,734.59,235.31,173.90
4,366.0,342.83,325.06,893.29,828.13,602.87,781.64,906.37,742.76,651.03,789.95,189.99


### 加载模型

In [12]:
from sklearn.linear_model import LinearRegression

In [13]:
def linear_regression_predict():

    # 提取训练集的特征变量和目标变量
    X_train = train_data.drop(columns=['vf_fr', 'vf_fr_s'])
    y_train_vf_fr = train_data['vf_fr']
    y_train_vf_fr_s = train_data['vf_fr_s']

    # 训练线性回归模型1：预测vf_fr
    model_vf_fr = LinearRegression()
    model_vf_fr.fit(X_train, y_train_vf_fr)

    # 训练线性回归模型2：预测vf_fr_s
    model_vf_fr_s = LinearRegression()
    model_vf_fr_s.fit(X_train, y_train_vf_fr_s)

    # 提取测试集的特征变量
    X_test = test_data

    # 进行预测
    vf_fr_predictions = model_vf_fr.predict(X_test)
    vf_fr_s_predictions = model_vf_fr_s.predict(X_test)

    # 将预测结果添加到测试数据中
    test_data['vf_fr_predicted'] = vf_fr_predictions
    test_data['vf_fr_s_predicted'] = vf_fr_s_predictions

    # 返回包含预测结果的DataFrame
    return test_data

In [14]:
predicted_data = linear_regression_predict()

In [16]:
print(predicted_data)
predicted_data.to_csv("F:/code/prediction/sample_submission.csv", index=False)

     pt_in  pt_out   tt_in  tt_out  vf_td_s   vf_td   vf_su  pt_in_s  \
0    928.0  696.36  335.77  423.94   803.92  450.25  311.88    32.05   
1    631.0  342.60  788.24  545.24   648.39  243.78  220.40   559.95   
2    683.0  181.68  877.73   84.23   754.15  768.92   10.62   803.87   
3    515.0  110.45  793.65   20.51   898.90  689.97  722.86   856.63   
4    366.0  342.83  325.06  893.29   828.13  602.87  781.64   906.37   
..     ...     ...     ...     ...      ...     ...     ...      ...   
197  779.0  956.51  203.05  863.97   748.05  384.54  377.33   307.82   
198  335.0   48.33   42.59  525.08     8.80  568.01  232.94   256.75   
199  211.0  876.00  877.48   11.36    22.32  160.17  671.74   755.93   
200  351.0  465.84   63.65  508.82   440.79  761.98  107.28   845.75   
201   79.0  246.59  680.63  315.77   701.38  417.81  841.35   208.37   

     pt_out_s  tt_in_s  tt_out_s  vf_su_s  vf_fr_predicted  vf_fr_s_predicted  
0       79.36   380.44    439.84   269.50       467.598

### 计算正确率

In [ ]:
def calculate_accuracy(test_file):
    # 读取包含预测结果的文件
    test_data = pd.read_csv(test_file)

    # 假设真实目标值存在于 test_data 中，列名为 vf_fr_true 和 vf_fr_s_true
    vf_fr_true = test_data['vf_fr_true']
    vf_fr_s_true = test_data['vf_fr_s_true']

    # 计算预测值和真实值之间的差异
    vf_fr_predicted = test_data['vf_fr_predicted']
    vf_fr_s_predicted = test_data['vf_fr_s_predicted']

    vf_fr_diff = vf_fr_true - vf_fr_predicted
    vf_fr_s_diff = vf_fr_s_true - vf_fr_s_predicted

    # 计算预测的准确率（例如，以百分比形式）
    vf_fr_accuracy = (1 - (vf_fr_diff.abs() / vf_fr_true.abs()).mean()) * 100
    vf_fr_s_accuracy = (1 - (vf_fr_s_diff.abs() / vf_fr_s_true.abs()).mean()) * 100

    return vf_fr_accuracy, vf_fr_s_accuracy